In [1]:
#%matplotlib widget
%pylab inline
pylab.rcParams['figure.figsize'] = (8, 3)
import  general.utils as utils
import result_analyse.resultloader
import result_analyse.visualisation as vs

from ipywidgets import interact, interactive, fixed, interact_manual,widgets

import pandas as pd
@interact
def result_selector(file=result_analyse.resultloader.get_runs()):
    if(file==None):return
    print('Analysing ',file)
    run_info,dataset,evalres=utils.loadState(file)
    stime=dataset.activity_events.iloc[0].StartTime
    etime=dataset.activity_events.iloc[-1].EndTime

    for i in range(len(evalres)):
            quality=evalres[i].quality
            print('Evalution quality fold=%d is %s' % (i, quality))
    print(len(dataset.sensor_events))
    
    vs.plot_CM(dataset,evalres)
    @interact
    def viewFold(fold= range(len(evalres))):
        @interact
        def view(start_date=widgets.DatePicker(value=pd.to_datetime(stime)),end_date=widgets.DatePicker(value=pd.to_datetime(etime))):
            duration=(pd.to_datetime(start_date),pd.to_datetime(end_date))
            real_events=vs.filterTime(evalres[fold].real_events,duration)
            pred_events=vs.filterTime(evalres[fold].pred_events,duration)
            vs.plotJoinAct(dataset,real_events,pred_events)
            vs.plot_per_act(dataset,{'test':evalres})
            @interact
            def view2(onlyAct=[(str(i)+" : "+dataset.activities[i],i)for i in dataset.activities_map]):
                print(onlyAct)
                #vs.visualize(dataset)
                # vs.my_result_analyse(evalres[i].real_events,evalres[i].pred_events)               
                real_events2,pred_events2=vs.remove_gaps(real_events,pred_events,onlyAct)
                vs.plotJoinAct(dataset,real_events2,pred_events2,onlyAct=onlyAct)
                vs.plotMyMetric(dataset,real_events,pred_events,onlyAct=onlyAct)
                vs.plotWardMetric(dataset,real_events,pred_events,onlyAct=onlyAct)



Populating the interactive namespace from numpy and matplotlib
Utils loaded successfully!!
metric  ==>  ['classical', 'CMbasedMetric', 'EventBasedMetric', 'event_confusion_matrix', 'metric_abstract', 'MyMetric']


interactive(children=(Dropdown(description='file', options=(('200206_23-46-58-A4H', '200206_23-46-58-A4H'), ('…